In [10]:
import pandas as pd
import numpy as np
from catboost import CatBoostClassifier
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

In [11]:
ybas = pd.read_csv('./data/res_baseline.csv', sep=",", index_col=0)
ybas = ybas['classification']
ybas

0       0
1       1
2       1
3       1
4       0
       ..
8941    0
8942    1
8943    1
8944    1
8945    0
Name: classification, Length: 8946, dtype: int64

In [12]:
data = pd.read_csv('processed_data.csv', sep=",")
data['Index + 1'][data['Index + 1'] >= 0] = 1
data['Index + 1'][data['Index + 1'] < 0] = 0

C:\Users\lucas\AppData\Local\Temp\ipykernel_3092\4204213129.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Index + 1'][data['Index + 1'] >= 0] = 1
C:\Users\lucas\AppData\Local\Temp\ipykernel_3092\4204213129.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Index + 1'][data['Index + 1'] < 0] = 0


In [13]:
data

,id_series,Index Name,Index - 9,Index - 8,Index - 7,Index - 6,Index - 5,Index - 4,Index - 3,Index - 2,...,list_speakers_ecb,list_speakers_fed,V_Cible_categ,polarity_score_ecb_LM,polarity_score_fed_LM,polarity_fed_LM,polarity_ecb_LM,sentiment_label_ecb_LM,sentiment_label_fed_LM,sentiment_label_LM
0,0,V2X Index,0.001045,0.005841,0.003832,-0.027519,-0.103565,-0.045086,-0.011265,0.005164,...,"['Philip R. Lane', 'Luis de Guindos']",['Vice Chairman Richard H. Clarida'],Hausse,"{'Positive': 81, 'Negative': 95, 'Polarity': -...","{'Positive': 57, 'Negative': 84, 'Polarity': -...",-0.191489,-0.079545,neutral,negative,negative
1,1,MOVE Index,-0.021497,0.007891,-0.013175,-0.008436,0.000000,0.026303,0.000556,0.001455,...,['Mario Draghi'],['Vice Chairman Stanley Fischer'],Hausse,"{'Positive': 78, 'Negative': 75, 'Polarity': 0...","{'Positive': 40, 'Negative': 96, 'Polarity': -...",-0.411765,0.019608,neutral,negative,negative
2,2,SPX Index,-0.001872,-0.008154,0.023588,0.004086,0.003493,0.003300,0.000885,-0.011304,...,"['Benoît Cœuré', 'Benoît Cœuré']","['Vice Chairman Stanley Fischer', 'Governor La...",Hausse,"{'Positive': 92, 'Negative': 142, 'Polarity': ...","{'Positive': 114, 'Negative': 255, 'Polarity':...",-0.382114,-0.213675,negative,negative,negative
3,3,SPX Index,0.004980,-0.000864,0.001677,0.000000,0.006030,-0.001083,0.000419,0.001492,...,"['Peter Praet', 'Peter Praet']",['Governor Jerome H. Powell'],Hausse,"{'Positive': 136, 'Negative': 136, 'Polarity':...","{'Positive': 71, 'Negative': 132, 'Polarity': ...",-0.300493,0.000000,neutral,negative,negative
4,4,EURUSD Curncy,0.000360,-0.001893,0.005579,-0.003056,-0.001171,-0.001623,-0.002350,-0.006444,...,['Yves Mersch'],['Chairman Jerome H. Powell'],Hausse,"{'Positive': 32, 'Negative': 55, 'Polarity': -...","{'Positive': 69, 'Negative': 140, 'Polarity': ...",-0.339713,-0.264368,negative,negative,negative
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8941,8941,SPX Index,-0.002045,-0.003642,0.008096,0.004949,-0.001264,0.000150,0.002482,0.000000,...,['Vítor Constâncio'],['Governor Daniel K. Tarullo'],Baisse,"{'Positive': 56, 'Negative': 63, 'Polarity': -...","{'Positive': 60, 'Negative': 170, 'Polarity': ...",-0.478261,-0.058824,neutral,negative,negative
8942,8942,SX5E Index,-0.027361,0.094386,0.002350,-0.005076,0.022555,0.000963,-0.060170,0.028323,...,['Lorenzo Bini Smaghi'],"['Chairman Ben S. Bernanke', 'Governor Randall...",Baisse,"{'Positive': 84, 'Negative': 196, 'Polarity': ...","{'Positive': 100, 'Negative': 198, 'Polarity':...",-0.328859,-0.400000,negative,negative,negative
8943,8943,SX5E Index,-0.008359,0.021658,-0.015986,-0.023528,0.020072,0.017822,0.005648,0.017045,...,"['Benoît Cœuré', 'Benoît Cœuré']","['Governor Jerome H. Powell', 'Governor Lael B...",Hausse,"{'Positive': 92, 'Negative': 142, 'Polarity': ...","{'Positive': 83, 'Negative': 158, 'Polarity': ...",-0.311203,-0.213675,negative,negative,negative
8944,8944,SRVIX Index,0.011945,0.002567,0.000000,0.005115,0.000364,0.003999,0.012978,0.012458,...,"['Peter Praet', 'Vítor Constâncio']",['Vice Chair Janet L. Yellen'],Baisse,"{'Positive': 131, 'Negative': 194, 'Polarity':...","{'Positive': 127, 'Negative': 252, 'Polarity':...",-0.329815,-0.193846,negative,negative,negative


In [14]:
X = data.drop(columns=['id_series', 'Index + 1', 'list_speakers_ecb', 'list_speakers_fed', 'V_Cible_categ', 'polarity_score_ecb_LM', 'polarity_score_fed_LM', 'id_ecb', 'id_fed'])
y = data['Index + 1']
y

0       1.0
1       1.0
2       1.0
3       1.0
4       1.0
       ... 
8941    0.0
8942    0.0
8943    1.0
8944    0.0
8945    1.0
Name: Index + 1, Length: 8946, dtype: float64

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [16]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6709 entries, 8582 to 7270
Data columns (total 16 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Index Name              6709 non-null   object 
 1   Index - 9               6709 non-null   float64
 2   Index - 8               6709 non-null   float64
 3   Index - 7               6709 non-null   float64
 4   Index - 6               6709 non-null   float64
 5   Index - 5               6709 non-null   float64
 6   Index - 4               6709 non-null   float64
 7   Index - 3               6709 non-null   float64
 8   Index - 2               6709 non-null   float64
 9   Index - 1               6709 non-null   float64
 10  Index - 0               6709 non-null   float64
 11  polarity_fed_LM         6709 non-null   float64
 12  polarity_ecb_LM         6709 non-null   float64
 13  sentiment_label_ecb_LM  6709 non-null   object 
 14  sentiment_label_fed_LM  6709 non-null

In [17]:
cat = CatBoostClassifier(
    iterations=100,
    learning_rate=0.1,
    cat_features=['Index Name', 'sentiment_label_ecb_LM', 'sentiment_label_fed_LM', 'sentiment_label_LM'],
    loss_function='Logloss',
    logging_level='Verbose',
    allow_writing_files=False
)
cat.fit(
    X=X_train,
    y=y_train,
    eval_set=(X_test,y_test),
    verbose = 10
)

0:	learn: 0.6889321	test: 0.6897752	best: 0.6897752 (0)	total: 152ms	remaining: 15s
10:	learn: 0.6678023	test: 0.6744865	best: 0.6744865 (10)	total: 217ms	remaining: 1.75s
20:	learn: 0.6595584	test: 0.6715549	best: 0.6715549 (20)	total: 279ms	remaining: 1.05s
30:	learn: 0.6528596	test: 0.6694467	best: 0.6694467 (30)	total: 349ms	remaining: 777ms
40:	learn: 0.6481592	test: 0.6694528	best: 0.6693957 (33)	total: 422ms	remaining: 608ms
50:	learn: 0.6434208	test: 0.6694804	best: 0.6693280 (47)	total: 486ms	remaining: 467ms
60:	learn: 0.6401414	test: 0.6699866	best: 0.6693280 (47)	total: 546ms	remaining: 349ms
70:	learn: 0.6369088	test: 0.6701143	best: 0.6693280 (47)	total: 605ms	remaining: 247ms
80:	learn: 0.6336627	test: 0.6700873	best: 0.6693280 (47)	total: 669ms	remaining: 157ms
90:	learn: 0.6294472	test: 0.6699868	best: 0.6693280 (47)	total: 733ms	remaining: 72.5ms
99:	learn: 0.6246973	test: 0.6704694	best: 0.6693280 (47)	total: 789ms	remaining: 0us

bestTest = 0.6693280032
bestIteratio

In [18]:
yp = cat.predict(X_test)

In [20]:
test = pd.merge(y_test, ybas, left_index=True, right_index=True)
yt = test['Index + 1'].to_numpy()
yb = test['classification'].to_numpy()
yp

array([1., 1., 1., ..., 1., 0., 1.])

In [21]:
from sklearn.metrics import accuracy_score, f1_score, recall_score, roc_auc_score
a = accuracy_score(yp, yt)
b = accuracy_score(yb, yt)
c = f1_score(yp, yt)
d = f1_score(yb, yt)
e = recall_score(yp, yt)
f = recall_score(yb, yt)
g = roc_auc_score(yp, yt)
h = roc_auc_score(yb, yt)

In [22]:
print(a, b)
print(c, d)
print(e, f)
print(g, h)

0.5462673223066608 0.4801072865444792
0.6121513183034009 0.4824210057854918
0.5527950310559007 0.5023169601482854
0.5435294952233818 0.4808648704022947


In [23]:
print('Accuracy : ', a)
print('F1-Score : ', c)
print('Recall : ', e)
print('AUC : ', g)

Accuracy :  0.5462673223066608
F1-Score :  0.6121513183034009
Recall :  0.5527950310559007
AUC :  0.5435294952233818
